In [591]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [592]:
url = "http://www.basketball-reference.com/draft/NBA_2016.html"
html = urlopen(url)

In [593]:
soup = BeautifulSoup(html, "html5lib")
type(soup)

bs4.BeautifulSoup

In [594]:
# soup.findAll('tr', limit=2)[1] # Find the first two, grab the second iteration
# soup.findAll('tr', limit=4)[3].findAll('th') # Skip first 3--groups 3 columns for mobile. Third 'tr' is for browser(check url source)

# Scrape the 'th' and create column headers with list comprehension
column_headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')[1:]]
column_headers


['Pk',
 'Tm',
 'Player',
 'College',
 'Yrs',
 'G',
 'MP',
 'PTS',
 'TRB',
 'AST',
 'FG%',
 '3P%',
 'FT%',
 'MP',
 'PTS',
 'TRB',
 'AST',
 'WS',
 'WS/48',
 'BPM',
 'VORP']

In [595]:
# Scrape the data rows but skip the first 2 'tr' which were column headers
data_rows = soup.findAll('tr')[2:]  # skip the first 2 header rows
type(data_rows)

list

In [596]:
# Get text out of data_rows
# player_data is a 2D matrix, and so needs a 2D list
# Nested list-comprehension
player_data = [[td.getText() for td in data_rows[i].findAll('td')[:]]
                for i in range(len(data_rows))]
# player_data

In [597]:
# Create the df
df = pd.DataFrame(player_data, columns=column_headers)
df.head(3)

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,3P%,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP
0,1,PHI,Ben Simmons,Louisiana State University,,,,,,,...,,,,,,,,,,
1,2,LAL,Brandon Ingram,Duke University,1,79,2279,740,316,166,...,.294,.621,28.8,9.4,4.0,2.1,-0.3,-.007,-3.8,-1.1
2,3,BOS,Jaylen Brown,University of California,1,78,1341,515,220,64,...,.341,.685,17.2,6.6,2.8,0.8,1.5,.053,-4.0,-0.7


In [598]:
# Find none values
df[df['Pk'].isnull()]

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,3P%,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP
30,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
31,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [599]:
# Assign a df subset to 'df' as a way of removing ^2 rows
df = df[df.Player.notnull()]

In [600]:
# Check your nulls
df[df['Pk'].isnull()]

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,3P%,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP


In [601]:
df.columns

Index(['Pk', 'Tm', 'Player', 'College', 'Yrs', 'G', 'MP', 'PTS', 'TRB', 'AST',
       'FG%', '3P%', 'FT%', 'MP', 'PTS', 'TRB', 'AST', 'WS', 'WS/48', 'BPM',
       'VORP'],
      dtype='object')

In [602]:
# Renaming columns with dictionary {'key/oldName:value/newName'}
df.rename(columns={'WS/48':'WS_per_48'}, inplace=True)
# df.rename(columns={'career_AST.1':'AST_per_g'}, inplace=True)

# Renaming columns by numbering duplicates
# cols=pd.Series(df.columns)
# for dup in df.columns.get_duplicates(): cols[df.columns.get_loc(dup)]=[dup+'.'+str(d_idx) if d_idx!=0 else dup for d_idx in range(df.columns.get_loc(dup).sum())]
# df.columns=cols

# Rename column values in for loop
df.columns.values[13:17] = [df.columns.values[13:17][col] + 
                                  "_per_G" for col in range(4)]

# Rename using built in string method .replace(), replace all occurances with x
df.columns = df.columns.str.replace('%', '_Perc')

df.columns

Index(['Pk', 'Tm', 'Player', 'College', 'Yrs', 'G', 'MP', 'PTS', 'TRB', 'AST',
       'FG_Perc', '3P_Perc', 'FT_Perc', 'MP_per_G', 'PTS_per_G', 'TRB_per_G',
       'AST_per_G', 'WS', 'WS_per_48', 'BPM', 'VORP'],
      dtype='object')

In [603]:
df.dtypes

Pk           object
Tm           object
Player       object
College      object
Yrs          object
G            object
MP           object
PTS          object
TRB          object
AST          object
FG_Perc      object
3P_Perc      object
FT_Perc      object
MP_per_G     object
PTS_per_G    object
TRB_per_G    object
AST_per_G    object
WS           object
WS_per_48    object
BPM          object
VORP         object
dtype: object

In [604]:
"""
originally str, .apply.to_numeric into floats, then fill NaNs with '0', before casting as int. 
"""
# Cleaning
# FG_Perc to VORP needs to be float. First .apply(pd.to_numeric)
df1 = df.loc[:,'FG_Perc':'VORP'].apply(pd.to_numeric, errors='coerce')
# then fill NaNs with 0, else NaNs won't convert to int
df1 = df1[:].fillna(0)

# For Yrs to AST (ints)
df2 = df.loc[:,'Yrs':'AST'].apply(pd.to_numeric, errors='coerce')
df2 = df2[:].fillna(0)
df2 = df2.loc[:,'Yrs':'AST'].astype(int)

# For Pk to College (strings)
df3 = df.loc[:,'Pk':'College']

print(df1.dtypes)
print(df2.dtypes)
print(df3.dtypes)


FG_Perc      float64
3P_Perc      float64
FT_Perc      float64
MP_per_G     float64
PTS_per_G    float64
TRB_per_G    float64
AST_per_G    float64
WS           float64
WS_per_48    float64
BPM          float64
VORP         float64
dtype: object
Yrs    int64
G      int64
MP     int64
PTS    int64
TRB    int64
AST    int64
dtype: object
Pk         object
Tm         object
Player     object
College    object
dtype: object


In [605]:
# Concatenate 
# Takes the (sorted) union of all df, join='outer'. This is the default option as it results in zero information loss.
df = pd.concat([df3, df2, df1], axis=1)
df.dtypes

Pk            object
Tm            object
Player        object
College       object
Yrs            int64
G              int64
MP             int64
PTS            int64
TRB            int64
AST            int64
FG_Perc      float64
3P_Perc      float64
FT_Perc      float64
MP_per_G     float64
PTS_per_G    float64
TRB_per_G    float64
AST_per_G    float64
WS           float64
WS_per_48    float64
BPM          float64
VORP         float64
dtype: object

In [606]:
# Insert column
# DataFrame.insert(loc, column, value, allow_duplicates=False)
df.insert(0, 'Draft_Yr', 2014)

# Drop column
# DataFrame.drop(labels, axis=0, level=None, inplace=False, errors='raise')
# df.drop('Rk', axis='columns', inplace=True) # Drop 'Rk' along column axis

In [607]:
df.columns

Index(['Draft_Yr', 'Pk', 'Tm', 'Player', 'College', 'Yrs', 'G', 'MP', 'PTS',
       'TRB', 'AST', 'FG_Perc', '3P_Perc', 'FT_Perc', 'MP_per_G', 'PTS_per_G',
       'TRB_per_G', 'AST_per_G', 'WS', 'WS_per_48', 'BPM', 'VORP'],
      dtype='object')

In [608]:
df.loc[:,'Tm':'College'] = df.loc[:,'Tm':'College'].astype(str)
df.dtypes

Draft_Yr       int64
Pk            object
Tm            object
Player        object
College       object
Yrs            int64
G              int64
MP             int64
PTS            int64
TRB            int64
AST            int64
FG_Perc      float64
3P_Perc      float64
FT_Perc      float64
MP_per_G     float64
PTS_per_G    float64
TRB_per_G    float64
AST_per_G    float64
WS           float64
WS_per_48    float64
BPM          float64
VORP         float64
dtype: object

In [609]:
df

,Draft_Yr,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,3P_Perc,FT_Perc,MP_per_G,PTS_per_G,TRB_per_G,AST_per_G,WS,WS_per_48,BPM,VORP
0,2014,1,PHI,Ben Simmons,Louisiana State University,0,0,0,0,0,...,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
1,2014,2,LAL,Brandon Ingram,Duke University,1,79,2279,740,316,...,0.294,0.621,28.8,9.4,4.0,2.1,-0.3,-0.007,-3.8,-1.1
2,2014,3,BOS,Jaylen Brown,University of California,1,78,1341,515,220,...,0.341,0.685,17.2,6.6,2.8,0.8,1.5,0.053,-4.0,-0.7
3,2014,4,PHO,Dragan Bender,,1,43,574,146,103,...,0.277,0.364,13.3,3.4,2.4,0.5,-0.3,-0.029,-4.3,-0.3
4,2014,5,MIN,Kris Dunn,Providence College,1,78,1333,293,166,...,0.288,0.610,17.1,3.8,2.1,2.4,0.1,0.004,-2.2,-0.1
5,2014,6,NOP,Buddy Hield,University of Oklahoma,1,82,1888,866,269,...,0.391,0.842,23.0,10.6,3.3,1.5,1.3,0.032,-2.7,-0.4
6,2014,7,DEN,Jamal Murray,University of Kentucky,1,82,1764,811,214,...,0.334,0.883,21.5,9.9,2.6,2.1,1.3,0.037,-2.6,-0.3
7,2014,8,SAC,Marquese Chriss,University of Washington,1,82,1743,753,348,...,0.321,0.624,21.3,9.2,4.2,0.7,1.8,0.050,-1.6,0.2
8,2014,9,TOR,Jakob Poeltl,University of Utah,1,54,626,165,165,...,0.000,0.544,11.6,3.1,3.1,0.2,1.6,0.125,-0.7,0.2
9,2014,10,MIL,Thon Maker,,1,57,562,226,114,...,0.378,0.653,9.9,4.0,2.0,0.4,1.3,0.113,-1.5,0.1


In [610]:
# Repeat the same for every year prior
url_template = "http://www.basketball-reference.com/draft/NBA_{year}.html"

In [611]:
draft_df = pd.DataFrame()

In [612]:
for year in range(1966, 2016):
    url = url_template.format(year=year)  # "http://www.basketball-reference.com/draft/NBA_{year}.html".format()
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html5lib')
    
    # Scrape player data
    data_rows = soup.findAll('tr')[2:] 
    player_data = [[td.getText() for td in data_rows[i].findAll('td')]
                for i in range(len(data_rows))]
    
    # Turn yearly data into a df
    year_df = pd.DataFrame(player_data, columns=column_headers)

    year_df.insert(0, 'Draft_Yr', year)
    
    # Append to the big dataframe
    draft_df = draft_df.append(year_df, ignore_index=True)

In [613]:
draft_df.tail()


,Draft_Yr,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,3P%,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP
6509,2015,56,NOP,Branden Dawson,Michigan State University,1,6,29,5,4,...,,1.000,4.8,0.8,0.7,0.0,0.0,.069,-6.6,0.0
6510,2015,57,DEN,Nikola Radicevic,,,,,,,...,,,,,,,,,,
6511,2015,58,PHI,J.P. Tokoto,University of North Carolina,,,,,,...,,,,,,,,,,
6512,2015,59,ATL,Dimitrios Agravanis,,,,,,,...,,,,,,,,,,
6513,2015,60,PHI,Luka Mitrovic,,,,,,,...,,,,,,,,,,


In [614]:
# Rename Columns
draft_df.rename(columns={'WS/48':'WS_per_48'}, inplace=True)
# Change % symbol
draft_df.columns = draft_df.columns.str.replace('%', '_Perc')

draft_df.columns.values[14:18] = [draft_df.columns.values[14:18][col] + 
                                  "_per_G" for col in range(4)]

draft_df.loc[:,'Yrs':'VORP'] = draft_df.loc[:,'Yrs':'VORP'].apply(pd.to_numeric, errors='coerce')

# Get rid of the rows full of null values
draft_df = draft_df[draft_df.Player.notnull()]
# Fill NaNs with 0
draft_df.loc[:] = draft_df.loc[:].fillna(0) # ASSIGN TO SELF

draft_df.loc[:, 'Yrs':'AST'] = draft_df.loc[:,'Yrs':'AST'].astype(int)

draft_df.dtypes

Draft_Yr       int64
Pk            object
Tm            object
Player        object
College       object
Yrs            int64
G              int64
MP             int64
PTS            int64
TRB            int64
AST            int64
FG_Perc      float64
3P_Perc      float64
FT_Perc      float64
MP_per_G     float64
PTS_per_G    float64
TRB_per_G    float64
AST_per_G    float64
WS           float64
WS_per_48    float64
BPM          float64
VORP         float64
dtype: object

In [615]:
draft_df

,Draft_Yr,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,...,3P_Perc,FT_Perc,MP_per_G,PTS_per_G,TRB_per_G,AST_per_G,WS,WS_per_48,BPM,VORP
0,1966,1,NYK,Cazzie Russell,University of Michigan,12,817,22213,12377,3068,...,0.000,0.827,27.2,15.1,3.8,2.2,51.7,0.112,-2.0,0.1
1,1966,2,DET,Dave Bing,Syracuse University,12,901,32769,18327,3420,...,0.000,0.775,36.4,20.3,3.8,6.0,68.8,0.101,0.6,8.5
2,1966,3,SFW,Clyde Lee,Vanderbilt University,10,742,19885,5733,7626,...,0.000,0.614,26.8,7.7,10.3,1.1,33.5,0.081,-2.4,-0.6
3,1966,4,STL,Lou Hudson,University of Minnesota,13,890,29794,17940,3926,...,0.000,0.797,33.5,20.2,4.4,2.7,81.0,0.131,0.1,5.9
4,1966,5,BAL,Jack Marin,Duke University,11,849,24590,12541,4405,...,0.000,0.843,29.0,14.8,5.2,2.1,59.3,0.116,-2.8,-1.4
5,1966,6,CIN,Walt Wesley,University of Kansas,10,590,10306,5002,3243,...,0.000,0.630,17.5,8.5,5.5,0.7,9.5,0.044,-2.8,-0.1
6,1966,7,LAL,Jerry Chambers,University of Utah,4,239,3691,1931,739,...,0.000,0.747,15.4,8.1,3.1,0.8,2.8,0.037,0.0,0.0
7,1966,8,BOS,Jim Barnett,University of Oregon,11,732,17410,8536,2259,...,0.000,0.797,23.8,11.7,3.1,3.0,33.2,0.092,-2.2,-0.3
8,1966,9,PHI,Matt Guokas,Saint Joseph's University,10,735,16603,4285,1446,...,0.000,0.727,22.6,5.8,2.0,3.0,26.7,0.077,-3.8,-2.2
9,1966,10,CHI,Dave Schellhase,Purdue University,2,73,513,208,76,...,0.000,0.567,7.0,2.8,1.0,0.8,-0.6,-0.061,0.0,0.0
